In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba
from molsim import mullerBrownPotential, mullerBrownPotentialAndGradient, plot_muller_brown_heatmap

# Exercise 3: Parallel Tempering

In parallel tempering we consider N systems. In each of these systems we perform a simulation in the canonical ensemble,
but each system is in a different thermodynamic state. Usually, but not necessarily, these states differ in temperature.
In what follows we assume that this is the case. Systems with a sufficiently high temperature pass all barriers in the
system. The low-temperature systems, on the other hand, mainly probe the local energy minima. The idea of parallel
tempering is to include MC trial moves that attempt to “swap” systems that belong to different thermodynamic states,
e.g., to swap a high temperature system with a low temperature system. If the temperature difference between the two
systems is very large, such a swap has a very low prob-ability of being accepted. This is very similar to particle
displacement in ordinary Monte Carlo.If one uses a very large maximum displacement a move has a very low probability of
being accepted. The solution to this problem is to use many small steps. In parallel tempering we use intermediate
temperatures in a similar way. Instead of making attempts to swap between a low and a high temperature, we swap between
systems with a small temperature difference. In principle the distribution of the position of a particle should be
symmetrical. The high-temperature system does show this symmetrical distribution.The total partition function of a system
with N canonical subsystems (Q) equals
$ Q = \Pi_{i=1}^{i=N} Q_i$

in which Qi is the canonical partition function of the individual system i

$ Q_i = \sum_{x_i} \exp [-\beta_i U(x_i)] $

where $\beta_i = 1/ (k_B T_i)$. For each of these systems, individual trial moves are performed. After a
randomly selected number of trial moves, an attempt is made to exchange configurations. Two
systems (i and j, $|i − j| = 1$) are selected at random, the systems are exchanged by choosing
$x_i (n) = x_j (o)$ and $x_j (n) = x_i (o)$. The ratio of acceptance probabilities equals

$\frac{acc(o \to n)}{acc(n \to o)} = \exp[(\beta_i - \beta_j) \times (U(x_i) - U(x_j))]$

Such trial moves will be accepted when there is enough overlap between the energies of systems
i and j. To demonstrate this technique, consider a two-dimensional system of a particle in a Muller-Brown potential. While a particle may have the thermal energy to visit another local minimum than the global minimum, it may not have the energy to overcome the barrier. Coupling this system to a system at a higher temperature that is able to sample the barrier region allows the system to visit all states that it should be able to reach.

## Question 1
Derive the acceptance criterium:
$\frac{acc(o \to n)}{acc(n \to o)} = \exp[(\beta_i - \beta_j) \times (U(x_i) - U(x_j))]$

from the detailed balance equation
$acc(o \to n) = \alpha(n \to o) \mathcal{N}(n)$


<font color="#7777ff">

## Answer
We want to derive the acceptance criterion for the parallel tempering swap. For this we will make use of the principals
of detailed balance. First we begin by defining the state of the old system and then the state of the new system.

Let us consider the "old" state first. System $o$ has the reciprocal temperature $\beta_i = \frac{1}{k_{B}T_i}$ and is at
configuration $x_i$ with energy $U(x_i)$.

The "new" state will be defined in a similar way. System $n$ has the reciprocal temperature $\beta_j =
\frac{1}{k_{B}T_j}$ and is at configuration $x_j$ with energy $U(x_j)$.

We choose two adjacent systems and propose a swap with some probability. We can assume that this swap can be symmetric
i.e. we can swap system $o$ with $n$ and the same probability of swapping system $n$ with $o$ meaning that:
$$
\alpha(o \rightarrow n) = \alpha(n\rightarrow o)
$$ 

We can then take the ratio of the acceptance probabilities to get:
$$
\begin{align*}
\frac{\operatorname{acc}(o \rightarrow n)}{\operatorname{acc}(n \rightarrow o)} = \frac{\alpha(n\rightarrow o)
\mathcal{N}(n)}{\alpha(o \rightarrow n) \mathcal{N}(o)} \\
\frac{\operatorname{acc}(o \rightarrow n)}{\operatorname{acc}(n \rightarrow o)} = \frac{\mathcal{N}(n)}{\mathcal{N}(o)}
\end{align*}
$$

Next we substitute the values given in the preamble for $\mathcal{N}(n)$ and $\mathcal{N}(o)$:
$$
\begin{align*}
    \frac{\operatorname{acc}(o \rightarrow n)}{\operatorname{acc}(n \rightarrow o)} &=
    \frac{\mathcal{N}(n)}{\mathcal{N}(o)} \\
    &= \frac{\exp[-\beta_i U(x_j) - \beta_j U(x_i)]}{\exp[-\beta_i U(x_i) - \beta_j U(x_j)]} \\
    &= \exp[ (-\beta_i U(x_j) - \beta_j U(x_i)) - (-\beta_i U(x_i) - \beta_j U(x_j)) ] \\
    &= \exp[ -\beta_i U(x_j) - \beta_j U(x_i) + \beta_i U(x_i) + \beta_j U(x_j) ]\\
    &\boxed{=\exp[ (\beta_i - \beta_j)(U(x_i) - U(x_j))]} 



\end{align*}

$$

## Question 2
First it is important to realize what the temperature does to the crossing of the barriers. Here we consider a density of states analysis for a very simple double well potential.

$U = x^4 - 4 x^2$

The probability density of being in a state (value of x) is calculated from the partition function

\begin{equation}
p = \frac{\exp(-\beta U(x))}{\int d x \exp(-\beta U(x))}
\end{equation}
For very low temperatures the system does not have enough thermal energy to cross the barrier. First please plot the potential energy surface.

Q1 What does the position distribution(x) (probability density) looks like at different temperature? 


Q2 If we define the transition region as $(-1 < x <1)$. What is the minimum temperature necessary to make it likely to see at
least 1000 transitions in a simulation of around $10^6$ steps?

<font color="#7777ff">

## Answer
Q1 Initally we see two sharp peaks at $-\sqrt{2}$ and $+\sqrt{2}$, however as the temperature increases the peaks flatten
out and approach a uniform distribution.

Q2 The minimum in the energy can be found at $\sqrt{2}$ and $-\sqrt{2}$ which correspond to an energy of $-4$, and the
maximum energy can be found at $x=0$ which corresponds to an energy of $0$. So there is a barrier of $\Delta U = 4$. If
we want to have $1000$ transitions in atleast $10^6$ then the probability of transition must atleast be $P_{trans} \geq
\frac{10^3}{10^6} = 10^{-3}$. We know that this probability must be proportional to the Boltzmann factor:
$$
\begin{align*}
    P_{trans} &\propto e^{\frac{\Delta U}{k_{B}T}} \\
    10^{-3} &\propto e^{\frac{-4}{k_{B} T}}\\
    -3\operatorname{ln}(10) &\propto \frac{4}{T} \\
    \frac{3}{4}\operatorname{ln}(10) &\propto T
\end{align*}
$$

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
x = np.linspace(-2.4, 2.4, 1000)

# arbitrary double well, which has minima at (-sqrt(2) and sqrt(2)) and a barrier of 4 k_BT

# start implementation
f = lambda x: x**4 - 4 * x**2
p = lambda x, beta: np.exp(-beta * f(x)) / np.trapezoid(np.exp(-beta * f(x)), x)
# end implementation


# plot the energy well
ax[0].plot(x, f(x), c="black", label=r"$U = x^4 - 4 x^2$")
ax[0].set_xlabel("X")
ax[0].set_ylabel(r"Energy / $k_BT$")
ax[0].legend()

# plot the density of states
numberOfBetas = 5
betaRange = (-1, 1)

cmap = plt.cm.coolwarm(np.linspace(1, 0, numberOfBetas))

for i, beta in enumerate(np.logspace(*betaRange, numberOfBetas)):
    ax[1].plot(x, p(x, beta), label=r"$T=$" + f"{1/beta:4.2f} " + r"$k_BT$", c=cmap[i])
ax[1].legend()
ax[1].set_xlabel("X")
ax[1].set_ylabel(r"Probability, $\frac{1}{\mathcal{Z}}e^{-\beta U}$")
# ax[1].set_yscale('log')

fig.tight_layout()

## Question 3

Now we move on to a 2D potential surface: muller brown potential. As shown in the following figure(please run the plot_muller_brown_heatmap() code) we defined a 2D potential surface with sereral local energy minimum( dark blue potential trap). The known energy minimum is loacted at [-0.557114228, 1.44889779]. Modify the code based on the results you got in question 1.

In [ ]:
plot_muller_brown_heatmap()

In [2]:
@numba.njit
def monteCarlo(
    temperatures: np.ndarray,
    numberOfCycles: int,
    parallelTemperingProbability: float = 0.0,
    maxDisplacement: float = 0.1,
):
    """
    Perform a Monte Carlo simulation of the Muller-Brown potential landscape with optional parallel tempering.

    Parameters
    ----------
    temperatures : np.ndarray
        An array of temperature values for each system. Shape: (numberOfSystems,).
    numberOfCycles : int
        The number of Monte Carlo cycles to perform.
    parallelTemperingProbability : float, optional
        The probability of attempting a parallel tempering swap at each cycle. Default is 0.0 (no parallel tempering).
    maxDisplacement : float, optional
        The maximum displacement scale for proposing a new position move. Default is 0.1.

    Returns
    -------
    positions : np.ndarray
        The array of positions for all systems across all cycles.
        Shape: (numberOfCycles, numberOfSystems, 2).

    Notes
    -----
    - Each system is simulated at a given temperature (from the input `temperatures`).
    - The function uses the Muller-Brown potential as the energy function.
    - Parallel tempering moves are attempted with a certain probability, which may help the simulation escape local minima.

    """
    numberOfSystems: int = len(temperatures)

    # Initialize arrays to store positions and energies
    # Using double precision floats for positions and energies
    positions = np.zeros((numberOfCycles, numberOfSystems, 2), dtype=np.float64)
    energies = np.zeros((numberOfCycles, numberOfSystems), dtype=np.float64)

    # Compute inverse temperatures (betas)
    # beta = 1 / T
    betas = [1.0 / T for T in temperatures]

    # Compute displacement scales based on temperatures, so that moves scale with sqrt(T)
    displacements = [np.sqrt(T) * maxDisplacement for T in temperatures]

    # Track acceptance rates for normal moves and parallel tempering moves (attempted, accepted) pairs per system
    translationAcceptance = np.zeros((numberOfSystems, 2), dtype=np.float64)
    parallelTemperingAcceptance = np.zeros((numberOfSystems - 1, 2), dtype=np.float64)

    # The known minimum position on the Muller-Brown surface to start from
    minimum = np.array([-0.557114228, 1.44889779], dtype=np.float64)

    # Initialize the first cycle positions at the known minimum
    # Repeat the minimum position for all systems
    positions[0] = np.repeat(minimum, numberOfSystems).reshape(2, numberOfSystems).T

    # Compute initial energies for all systems
    for system in range(numberOfSystems):
        # mullerBrownPotential is assumed defined elsewhere
        energies[0, system] = mullerBrownPotential(positions[0, system, 0], positions[0, system, 1])

    # Main Monte Carlo loop
    for cycle in range(1, numberOfCycles):
        # Attempt position updates for each system
        for system in range(numberOfSystems):
            # Propose a new position by random displacement
            translationAcceptance[system, 0] += 1
            newPosition = positions[cycle - 1, system] + displacements[system] * (np.random.rand(2) - 0.5)
            newEnergy = mullerBrownPotential(newPosition[0], newPosition[1])

            # Metropolis acceptance criterion
            if np.random.rand() < np.exp(-betas[system] * (newEnergy - energies[cycle - 1, system])):
                # Accept the move
                positions[cycle, system] = newPosition
                energies[cycle, system] = newEnergy
                translationAcceptance[system, 1] += 1
            else:
                # Reject the move, keep old position and energy
                positions[cycle, system] = positions[cycle - 1, system]
                energies[cycle, system] = energies[cycle - 1, system]

        # Attempt parallel tempering swap with a given probability
        if np.random.rand() < parallelTemperingProbability:
            systemA = np.random.randint(0, numberOfSystems - 1)
            systemB = systemA + 1

            parallelTemperingAcceptance[systemA, 0] += 1  # Count the swap attempt

            #!##!##!##!# start implementation
            # Acceptance criterion for parallel tempering
            if np.random.rand() < np.exp(
                (betas[systemB] - betas[systemA]) * (energies[cycle, systemB] - energies[cycle, systemA])
            ):
                #!##!##!# end implementation
                parallelTemperingAcceptance[systemA, 1] += 1  # Count the successful swap

                # Swap positions
                tmp_pos = positions[cycle, systemB].copy()
                positions[cycle, systemB] = positions[cycle, systemA]
                positions[cycle, systemA] = tmp_pos

                # Swap energies
                tmp_en = energies[cycle, systemB]
                energies[cycle, systemB] = energies[cycle, systemA]
                energies[cycle, systemA] = tmp_en

    # Print acceptance rates for debugging or analysis
    print("Translation acceptance rates:", translationAcceptance[:, 1] / translationAcceptance[:, 0])
    if parallelTemperingProbability > 0.0:
        print(
            "Parallel tempering acceptance rates:",
            parallelTemperingAcceptance[:, 1] / parallelTemperingAcceptance[:, 0],
        )

    return positions

In this exercise we will simulate with a Monte Carlo sampler. Use the code below to run a set of temperatures. The Monte Carlo code can run multiple simulations at once for different temperatures, which will be important for later. For now, you can use that to test multiple different temperatures. Try some temperatures and look at the results with the visualization provided below (note that the visualization now plots the 4 different trajectories).

What is, empirically, the minimal temperature necessary to see some transitions accross to the minima found at (0.0, 0.5) and (0.8, 0.0)?

**Note:** theoretically, at any temperature larger than 0 we will see a transition at infinite sampling time. For the
current amount of samples ($2 \cdot 10^7$) it is just very, very unlikely.

<font color="#7777ff">

## Answer

When the energy is 8.125, if the atoms transition from the starting minima to the minima at (0.5, 0) then they will also
transition to the minima at (0.8, 0)

In [ ]:
# Define the number of Monte Carlo cycles and the temperatures for each system
numberOfCycles = int(2e7)
# start refactor
temperatures = [6.875, 8.125] # 6.25, 8.75
# end refactor

# Run the Monte Carlo simulation
positions = monteCarlo(temperatures, numberOfCycles, parallelTemperingProbability=0.0)

<font color="#7777ff">

## Student note

I had only enough free RAM to run 2 simulations together. So that Is why I had to modify the code below

In [ ]:
# Plot the Muller-Brown potential landscape and the positions of the systems
fig, ax = plt.subplots(1, 2, figsize=(12, 10))
ax = ax.flatten()
for i in range(2):
    plot_muller_brown_heatmap(ax[i])
    ax[i].scatter(*positions[:, i].T, s=0.5, c="red", label=f"T={temperatures[i]}")
    ax[i].set_title(f"T={temperatures[i]}", c="red")
fig.tight_layout()

## Question 4
Now that you have derived the detailed balance equation, try to understand the following Monte Carlo algorithm. What two moves happen each cycle? What parameters can we set? How do we specify whether we do parallel tempering?

If you look at the acceptance criterium, the proposed parallel tempering move is currently never accepted due to the wrong acceptance criterium `np.random.rand() < 0.0`. Implement the correct acceptance criterium and rerun the simulation with a non-zero parallel tempering probability. 

Do you see exchange acceptance rates?
What is the approximately the minimal temperature to sample point in the minimum (0.8, 0.0)?
What is the approximately the minimal temperature to sample point in the minimum (0.0, 0.5)?

<font color="#7777ff">

## Answer
At every step we do a monte carlo simulation in parallel across all systems. This proceeds using the normal acceptance
criteria for this type of simulation. Secondly at each step we decide whether we want to swap the configurations of two
systems. 

We can set the temperature, the number of cycles and the probability of performing parallel tempering. We can specify
parallel tempering by setting ``parallelTemperingProbability``$\geq 0$. The acceptance rates look roughly similar.

To sample the point in the minimum at (0.0, 0.5) I had to set the temperature to 4.875. 
To sample the point in the minimum at (0.8, 0.0) I had to set the temperature to

In [ ]:
# start refactor

# T = [3.875, 4.875]
#Translation acceptance rates: [0.45037172 0.45202532]
#Parallel tempering acceptance rates: [0.88295377]

numberOfCycles = int(2e7)
temperatures = [4.875, 5.375] # 4.875, 6.875
pt_probability = 0.2 
ptPositions = monteCarlo(temperatures, numberOfCycles, parallelTemperingProbability=pt_probability)
# end refactor

Translation acceptance rates: [0.45217667 0.45301202]
Parallel tempering acceptance rates: [0.94983386]


: 

In [1]:
# Plot the Muller-Brown potential landscape and the positions of the systems
fig, ax = plt.subplots(1, 2, figsize=(12, 10))
ax = ax.flatten()
for i in range(2):
    plot_muller_brown_heatmap(ax[i])
    ax[i].scatter(*ptPositions[:, i].T, s=0.5, c="red", label=f"T={temperatures[i]}")
    ax[i].set_title(f"T={temperatures[i]}", c="red")
fig.tight_layout()

NameError: name 'plt' is not defined